# SIXT33N Phase 2: System Identification

### EECS 16B: Designing Information Devices and Systems II, Spring 2021

Zhongkai Wang (2021)

zhongkai@berkeley.edu

## Table of Contents

* [Part 0: Introduction](#intro)
* [Part 1: Power and Actuator Circuits](#part1)
* [Part 2: Open Loop Data Collection and Parameter Estimation](#part2)
* [Part 3: Determining the Operating Point](#part3)

## <span style="color:#ba190f"> You need to run the python scripts one by one, or errors will show up as they rely on the variables defined above!!


<a id='intro'></a>
## <span style="color:blue">Part 0: Introduction

In this phase, we will
    
- build the SIXT33N actuators (motor controller)
- test the encoders to verify our sensing readings are accurate.
- model the the car as an open loop system.
- collect data and determine the parameters of our model using least-squares.


<a id='part1'></a>
## <span style="color:blue">Part 1: Power and Actuator Circuits</span>

### Virtual Materials
- 2 NPN BJTs (Bipolar Junction Transistor)
- 2 Diodes
- Multimeter
- Oscilloscopes
- Arduino Microcontroller
- 1 1.5k$\Omega$ resistors
- 1 4k$\Omega$ resistors
- 1 3-termial switch

### Tasks

### 1.0 Plan Your Breadboard Layout

Our circuit will need two sources of power (as regulators will slow down the simulation, we use the powers directly from battery and Arduino board):

- 9V (Will come from the battery) 
- 5V (Will come from the Arduino board) 

Try to divide your breadboard into the following sections so that you have room for them and the later of the project. 

- Use top (+) rail for 9V, and bottom (+) rail for 5V.
- Connect the two (-) to create the ground rail.
- The purple sections are reserved to build the motor controller circuits.

<img style="width:700px" src="images/floorplan.png">

The power distribution is shown in the diagram below. The motor controller circuits use the 9V supply from battery to drive the motor. The encoder of the motors gets 5V supply from the Arduino board. **Note that all the grounds (including the grounds of the battery, the Arduino board and the breadboard) are connected.**

<br/><br/>

<center>
<img width="500px" src="images/proj-power-2.png">
</center>

### 1.1 Get familiar with the motor encoder

As mentioned in project phase 1, the motor has 6 pins. The two leftmost pins are the driving pins that you connect using motor controller circuit. The other 4 pins are for the encoder module, which outputs a single pulse on channel A and B whenever the motor rotates for a specific amount. By counting the number of pulses outputted by the encoder, we can easily gauge how fast the motor is rotating and whether adjustments need to be made to the input signal. In this phase, we will use **Channel A** to get the rotating speed of the motors.

<img width="400px" src="images/motor_with_encoder.png">


### 1.2 Build the circuit

For this lab, you have two choices - you can start with circuit you made in project phase 1, or you can use the starter circuit and re-construct the ciruit in TinkerCad.

####  <span style="color:#ba190f"> 1.2.1 If you use the project phase 1 circuit, you need to make the following changes. 

1. Make a duplicate of your circuit from project phase 1 in TinkerCad.
    
    - Reconstruct the power supplies as shown in the floorplan figure. Connect breadboard top (+) rail to battery and bottom (+) rail to Arduino 5V supply pin. Remember to connect the GNDs of battery, breadboard and Arduino.
    - **Remove the 5V and 3.3V regulators from your circuit.** The regulars will slow down the simulation and change the results.
    - **Remove your connections connecting Arduino pin 9 and controller circuits from phase 1.**
    
    
2. For the motor controller circuit,

    - **Connect $1.5k \Omega$ resistor for the left BJT base and $4k \Omega$ to the right BJT base.** Thus, we intentially add some differences between the left and right controller circuits, and make their rotational speed different. 

    
3. Connect the motor encoders for **each wheel**.

    - Connect both the **encoder grounds (pin 3 in the motor diagram)** to breadboard ground rail (-).
    - Connect both the **encoder powers (pin 6 in the motor diagram)** to bottom (+) rail (5V power rail).
    - Connect the **channel A (pin 5 in the motor diagram)** pins of the <span style="color:#ba190f">left motor</span> and <span style="color:#ba190f">right motor</span> to Arduino <span style="color:#ba190f"> pin 3</span> and <span style="color:#ba190f">pin 2</span> respectively.

    
4. Connect Arduino to controller circuits.

    - Connect the node labeled **Arduino output pin** of the <span style="color:#ba190f">left motor</span> controller circuit to <span style="color:#ba190f">pin 10</span> on the Microcontroller. 
    - Connect the node labeled **Arduino output pin** of the <span style="color:#ba190f">right motor</span> controller circuit to <span style="color:#ba190f">pin 9</span> on the Microcontroller. 

    
#### <span style="color:#ba190f">1.2.2 If you use the [starter circuit](https://www.tinkercad.com/things/5bCH3zNDtGS ) in TinkerCAD for the following lab, the procedure is the following.

1. Review phase 1 of the project. Construct the circuit below for **each wheel** using motors, resistors, NPN BJTs and diodes. Please note that

    - The motor is reversible. If it turns out to go backwards later, just reverse the two terminals of the motor.
    - Make sure to match the Base, Emitter, and Collector of the BJT properly to the schematic.
    - Add a diode in parallel with the motor. Be careful which terminal is attached to the 9V power supply.
    - Add a switch between 9V and the motor drivers. You only need ONE switch!
    - **Please connect $1.5k \Omega$ resistor for the left BJT base and $4k \Omega$ to the right BJT base.** Thus, we intentially add some differences between the left and right controller circuits, and make their rotational speed different. 

   <img width="250px" src="images/motor-drive-with-switch.png">

2. Connect the motor encoders for **each wheel**.

    - Connect both the **encoder grounds (pin 3 in the motor diagram)** to breadboard ground rail (-).
    - Connect both the **encoder powers (pin 6 in the motor diagram)** to bottom (+) rail (5V power rail).
    - Connect the **channel A (pin 5 in the motor diagram)** pins of the <span style="color:#ba190f">left motor</span> and <span style="color:#ba190f">right motor</span> to Arduino <span style="color:#ba190f"> pin 3</span> and <span style="color:#ba190f">pin 2</span> respectively.
    

3. Connect Arduino to controller circuits.

    - Connect the node labeled **Arduino output pin** of the <span style="color:#ba190f">left motor</span> controller circuit to <span style="color:#ba190f">pin 10</span> on the Microcontroller. 
    - Connect the node labeled **Arduino output pin** of the <span style="color:#ba190f">right motor</span> controller circuit to <span style="color:#ba190f">pin 9</span> on the Microcontroller. 


### 1.3 Test the circuit.

1. Make sure parameter **`MODE`** in line 6 being 0. **If you used your previous circuit, you need to upload `dynamics_data.ino` to your code environment and load in onto the microcontroller.** 

2. Open the serial monitor.
 
     - As the program runs, you should see the "Encoder values" in the serial monitor. This is the number of encoder pulses within every 200ms duration. **The two numbers are for the left motor and right motor, respectively.** The numbers will stablize after about five cycles. 
     - To control the PWM duty cycle, you can change parameter `PWM_CONST` in line 7 of the code. **When `PWM_CONST` is 200, encoder value of the left motor is around 359.**

    
3. You can also try to debug your circuit by  

    - probing the two encoder outputs (pin **channel A** ) with oscilloscopes and observe the encoder outputs. If you set the `Time Per Division` to 1ms, you should see the following waveform (`PWM_CONST` is 200).
    - observing the motor rpm written on it when the simulation is running.

<img width="350px" src="images/encoder_output.png">

## Questions

**<span style="color:#075a04"> 1. Which motor rotates at a higher speed?</span>**

- The left motor
- The right motor
- The two motors have almost same rotating speed

< YOU ANSWER ON GRADESCOPE >

<span style="color:#075a04"> **2. In real life, the two motors might rotate at different rotating speeds. What could cause the difference in our circuit? Choose all the answers that are correct.** 

- The motors may have different sensitivities to current, due to electrical or mechanical difference.
- The BJTs may have different output currents even with same input voltages.
- The two resistors connecting to BJT bases may have resistance errors.
- The PWM duty cycles from the Arduino may be inaccurate.

< YOU ANSWER ON GRADESCOPE >

<span style="color:#075a04"> **3. Change parameter `PWM_CONST` to 255, what is the number of encoder pulses you get for the left motor from the Serial Monitor (after the values stabilize)? <span style="color:#ba190f"> Enter integer value (e.g. 100).** 

< YOU ANSWER ON GRADESCOPE >
 
<span style="color:#075a04"> **4. When increasing parameter `PWM_CONST`, how does the period of the encoder pulses change? The period** 
    
- increases
- decreases
- does not change

< YOU ANSWER ON GRADESCOPE >
    
<span style="color:#075a04"> **5. Change parameter `PWM_CONST` to 150, what is the difference of the number of encoder pulses between the left and right motors (after the values stabilize)? Use the number of the left motor minus the number of the right motor. <span style="color:#ba190f"> Enter integer value (the number could be negative, e.g. -100).** 

< YOU ANSWER ON GRADESCOPE >

<a id='part3'></a>
## <span style="color:blue">Part 2: Open Loop Data Collection and Parameter Estimation</span>




### 2.0. Please read the [lab note](https://drive.google.com/file/d/1JTsD3TQo6t1tnkJU322V40ZpRp2avbUA/view?usp=sharing).

Before trying to control SIXT33N, we will first determine the system operating point: since your motors are not identical, we need to find an operating velocity that both motors can reach. We will assume that velocity varies approximately linearly with applied voltage, so we will collect data across a range of applied voltages and then perform a least-squares linear regression on a subset of the data (located around your operating point). You will do this separately for each wheel. By doing this, we are effectively forming a simplified model of your car (you will use a more detailed model to design your control scheme, but this is enough for us to get the data we need).

### 2.1. Collect data

Now, you will collect the data. Glance through the code - when parameter **`MODE`** is 1, it records the position of each wheel while varying the PWM input signal (noted as $u[k]$) linearly from HIGH_PWM to LOW_PWM and back up. The sampling period is $T_s = 200\mathrm{ms}$. A long sampling period is used to minimize error due to quantization, the rounding error from measuring only integer encoder ticks.

The five parameters in code block `SID1` will sweep through the whole range of PWM values, from maximum to minimum and back to maximum, while collecting only 1 sample per PWM. This means **the motor will stop for a short time in the middle of the test.**

#### About DATA collection:
1. Change parameter **`MODE` to 1**. 
2. Clear the Serial Monitor.
3. **Make sure the switch is ON before starting the simulation.** When running **`dynamics_data.ino`**, Arduino will run the motors and collect data, then read data collected onto the Serial Monitor. You need to wait for a while for the data to show up, it takes about 10 seconds of simulator time.
4. When reading the data from the Serial Monitor:
    - The data is printed in lines as comma separated 3-tuples. 
    - Copy and paste, ONLY these 3-tuples, nothing else.
    - Your data should look something like this (albeit with more data points):
<img width="400px" src="images/data_coarse_example.png">

### 2.2. Plot data:

- Once you've copied data from the serial monitor, paste it into a text file called **`data.txt`** in the *home directory* of this lab. **Delete all the empty lines and useless messages, only keep the numbers as picture above.**

The example plot below shows an example of extreme differences between the two wheels from **hands-on lab**. Here, the 1.5$\mathrm{k\Omega}$ resistors at the bases of the BJT's were replaced by 1$\mathrm{k\Omega}$ (left) and 3$\mathrm{k\Omega}$ (right) resistors. The smaller resistor drives the BJT with more current ($I=\frac{V}{R}$, smaller $\downarrow R\implies \uparrow I$), and oppositely for the larger resistor. This mimics a car with a much stronger left wheel than right wheel. The plot shows the drastic difference in response between the two wheels, and how the wheels behave when slowing to 0 versus speeding up from 0.

<img width="400px" src="images/example_coarse_data.png">


In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
with open('data.txt', 'r') as file:
    data = []
    for line in file.readlines():
        if not len(line.strip()) == 0 : 
            data.append ([int(num) for num in line.strip().split(',')])
    data = np.array(data)

# compute velocity from distances
vleft  = data[:, 1]
vright = data[:, 2]
u = data[:, 0]    # duty cycle     #####.reshape((-1, 1)) 

left_des, left_asc = np.array_split(vleft, 2)
right_des, right_asc = np.array_split(vright, 2)
u_des, u_asc = np.array_split(u, 2)

plt.plot(u_des, left_des, 'b-',  u_des, right_des, 'y-')
plt.plot(u_asc, left_asc, 'b-o',  u_asc, right_asc, 'y-^')
plt.xlabel("u (input via PWM)")
plt.ylabel("Velocity of Wheels")
labels = ("left (descending)", "right (descending)",
          "left (ascending)" , "right (ascending)")
plt.legend(labels, loc=0)
plt.show()

In real life, the car only has to run within a small range of velocities due to the non-linearity, so we need to collect more samples at each PWM, over a smaller PWM range. However, from the plot above, the simulated data are good enough, and we can skip this step and find a linear approximation for the behavior of the motors directly. 

## Questions

<span style="color:#075a04"> **6. What is the slope of line under the *right ascending case* from the data or plot above? You can estimate it from the first and last points of the line. Hint: Use print() function to print out the values you need. <span style="color:#ba190f">Enter numerical value with two decimal points (e.g. 3.14).**
    
< YOU ANSWER ON GRADESCOPE >

### 2.3. Least-Squares Regression
Now that we have some data, we can try performing least-squares regression.

1. Write a function that takes the data as parameters, performs least squares, and extracts the parameters. The function [`np.linalg.lstsq`](https://het.as.utexas.edu/HET/Software/Numpy/reference/generated/numpy.linalg.lstsq.html) and [`np.vstack`](https://het.as.utexas.edu/HET/Software/Numpy/reference/generated/numpy.vstack.html )will be helpful here. **From the [note](https://drive.google.com/file/d/1JTsD3TQo6t1tnkJU322V40ZpRp2avbUA/view?usp=sharing), in our linear regression method, we assume that $v=\theta u-\beta$, and the single matrix-vector equation of the form $Ax = b$ is shown as follows. However, the example in the webpage uses $v=\theta u+\beta$.** Please think about how to modify the example and use it here.
2. Call the function once for each wheel.
3. Record the values of $\theta$ and $\beta$ for each wheel.

<img width="400px" src="images/fomular.png">

In [ ]:
# Write a function that formulates and performs least squares
# and returns theta and beta for one wheel
def identify_parameters(u, v):
    # Returns (theta, beta) for given (u, v)
    ####### YOUR CODE STARTS HERE ###############
    
    return pass
    ####### YOUR CODE ENDS HERE #################


# extract parameters, call your function here
theta_left, beta_left = identify_parameters(u, vleft) # 0, 0
theta_right, beta_right = identify_parameters(u, vright) # 0, 0

print("float theta_left = {:.4g};".format(theta_left))
print("float theta_right = {:.4g};".format(theta_right))
print("float beta_left = {:.4g};".format(beta_left))
print("float beta_right = {:.4g};".format(beta_right))

In [ ]:
# plot results of least squares fit 
u = u.reshape(-1)
vleft_LS = theta_left*u-beta_left
vright_LS = theta_right*u-beta_right
plt.plot(u, vleft, 'bo',  u, vright, 'yo', u, vleft_LS, 'b-', u, vright_LS, 'y-')
plt.xlabel("u (input via PWM)")
plt.ylabel("Velocity of Wheels")
plt.legend(("left", "right"), loc=0)

## Questions

<span style="color:#075a04"> **7. What is the $\theta$ you get for the *left* motor from the least-squares regression?. <span style="color:#ba190f">Enter numerical value with two decimal points (e.g. 3.14).**
    
< YOU ANSWER ON GRADESCOPE >

### 2.4. Evaluate Your Fit

To test that we have estimated the parameters accurately, we will simulate the model using the parameters we have found. When we compare the simulated data with the collected data, we expect them to match up pretty well. 
- **Please visualize or think about why the distance plot is curved the way it is.**

In [ ]:
# least-squares regression
def sim(d0, u, theta, beta):
    d = np.zeros(len(u)+1)
    d[0] = d0
    for t in range(len(u)):
        d[t+1] = d[t] + theta*u[t] - beta
    return d

# simulation
def get_distances_from_velocities(v):
    d = np.zeros(len(u) + 1)
    d[0] = 0
    for t in range(len(u)):
        d[t+1] = d[t] + v[t] 
    return d
        
# plot simulated trajectories 
dleft = get_distances_from_velocities(vleft)
dright = get_distances_from_velocities(vright)

dleft_regression  = sim(0, u, theta_left, beta_left)
dright_regression = sim(0, u, theta_right, beta_right)
plt.plot(dleft, 'b.', 
         dright, 'y.',
         dleft_regression, 'b-', 
         dright_regression, 'y-',
        )
plt.xlabel("time")
plt.ylabel("distance")
plt.legend(("left (sim)", "right (sim)", "left (regression)", "right (regression)"), loc='upper left')

## Questions

<span style="color:#075a04"> **8. What is the final distance of the *right* motor from least-squares regression model at the end of the time? Hint: Use print() function to print out the values you need. <span style="color:#ba190f"> Enter numerical value with two decimal points (e.g. 3.14).**
    
< YOU ANSWER ON GRADESCOPE >
    

<a id='part3'></a>
## <span style="color:blue">Part 3: Determining the Operating Point</span>

In hands-on lab, each wheel may have a different range of velocities, illustrated here.

<img src="./images/partial_overlap_lsq.png">


In order to drive straight, the car must be operating at a velocity achievable by both wheels. A good choice of target velocity is the midpoint of the overlapping range of velocity. The below cell will calculate this.

In [ ]:
min_vel = max(min(vleft_LS), min(vright_LS))
max_vel = min(max(vleft_LS), max(vright_LS))
print('Velocity range = [{:0.3f}, {:0.3f}]'.format(min_vel, max_vel))
midpoint = (min_vel+max_vel)/2
print('\nOperating point:\nfloat v_star = {:.3f};'.format(midpoint))

u = u.reshape(-1)
vleft_LS = theta_left*u-beta_left
vright_LS = theta_right*u-beta_right
plt.plot(u, vleft_LS, 'b-', u, vright_LS, 'y-')
for i in (min_vel, max_vel):
    plt.plot(u, 0*u + i, 'g-')
plt.plot(u, vleft, 'bo',  u, vright, 'yo')
plt.xlabel("u (input via PWM)")
plt.ylabel("Velocity of Wheels")
plt.legend(("left", "right", "overlap"), loc=0)

## Questions

<span style="color:#075a04"> **9. What is the operating point of velocity you get from the procedure above? <span style="color:#ba190f">Enter numerical value with two decimal points (e.g. 3.14).**
    
< YOU ANSWER ON GRADESCOPE >

<img width='30px' align='left' src="http://inst.eecs.berkeley.edu/~ee16b/sp16/lab_pics/check.png">
<br/>

## <span style="color:green">CHECKOFF</span>

- Complete the Project: Front End assignment on Gradescope.
- Make sure your circuits are saved properly in Tinkercad.